In [1]:
from selenium.webdriver.chrome.service  import Service
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import json
import pandas as pd
import time
from ordered_set import OrderedSet


In [2]:
service = Service()
driver = webdriver.Chrome(service = service)


# 1. Login to Instagram Handle

In [2]:
driver.get("https://www.instagram.com")
time.sleep(3)

userNameInput = driver.find_element("name","username")
userNameInput.send_keys("SAMPLE USERNAME")

passwordInput = driver.find_element("name","password")
passwordInput.send_keys("SAMPLE PASSWORD")

loginButton = [i for i in driver.find_elements("tag name","button")
   if(i.text=="Log in")
][0]

loginButton.click() 
time.sleep(8.5)


driver.get("https://www.instagram.com")
time.sleep(2.5)

disableNotification = driver.find_element('xpath','/html/body/div[3]/div[1]/div/div[2]/div/div/div/div/div[2]/div/div/div[3]/button[2]')
disableNotification.click()


# 2. Searching A keyword (Food)

In [116]:
#find and click search icon
searchIcon = driver.find_element('xpath','/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[1]/div/div/div/div/div[2]/div[2]/span/div/a')
searchIcon.click()

# find search input and type food
searchInput = [ i for i in driver.find_elements('tag name','input')
    if i.get_attribute("placeholder")=='Search'
][0]
searchInput.send_keys('food')

#find suggestionList
suggestionList = driver.find_element("xpath",'/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[1]/div/div/div[2]/div/div/div[2]/div/div/div[2]')
suggestionsHtml = suggestionList.get_attribute('outerHTML') #fetch suggestionList HTML

data = BeautifulSoup(suggestionsHtml,'html.parser') #parse html using beautifulSoup

#fetch handle names
handleNames = data.select('a div:nth-child(1)>div>div>div:nth-child(2)>div>div:nth-child(1)')

#print handle names
print("Suggestions list Handle Names : \n")
for i in handleNames:
   
    x = i.select('span:nth-child(1)')
    if(x[0].get_text()[0]!="#"):
        print('\t',i.select('span span')[0].get_text().split('•')[0])


# 3. Searching and Opening a Profile

In [5]:
def searchAndOpenProfile(profile_name):
    
    driver.get("https://www.instagram.com") #go to home page
    
    # find and click search icon
    searchIcon = driver.find_element('xpath','/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[1]/div/div/div/div/div[2]/div[2]/span/div/a')
    searchIcon.click()

    # find search input and search profile_name
    searchInput = [ i for i in driver.find_elements('tag name','input')
        if i.get_attribute("placeholder")=='Search'
    ][0]
    searchInput.send_keys(profile_name)

    #open searched profile
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[1]/div/div/div[2]/div/div/div[2]/div/div/div[2]/div/a[1]"))
    ).click()
    
    time.sleep(2.5)


In [120]:
profile_name = 'So Delhi'
searchAndOpenProfile(profile_name)


# 4. Follow/Unfollow given Handle

In [1]:
def followAccount():

    #find follow button
    btn = WebDriverWait(driver, 5).until(EC.presence_of_element_located(("xpath","/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/div[2]/section/main/div/header/section/div[1]/div[1]/div/div[1]/button")))
    
    if(btn.text=='Follow'):
        btn.click()
    else:
        print('already followed')


In [2]:
def unfollowAccount():
    
    # find following button
    btn = WebDriverWait(driver, 5).until(EC.presence_of_element_located(("xpath","/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/div[2]/section/main/div/header/section/div[1]/div[1]/div/div[1]/button")))
    
    if(btn.text=='Following'): # if following
        btn.click()
        time.sleep(1.6)
       
        # find unfollow button and click
        unfollowBtn = [i for i in WebDriverWait(driver,5).until(EC.presence_of_all_elements_located(("tag name",'span')))
         if i.text=='Unfollow'
        ][0]
        
        unfollowBtn.click()
    else:
        print('already Unfollowed')


# part(1):- Open instagram handle of 'So Delhi'

In [242]:
profile_name = 'So Delhi'
searchAndOpenProfile(profile_name) # function call searchAndOpenProfile()


# part(2):- Start following it

In [243]:
followAccount()   # call followAccount() function
time.sleep(2.5)


already followed


# part(3):- Unfollow the account

In [244]:
unfollowAccount()  # call unfollowAccount() function

# 5. Like/Unlike Posts

In [42]:
def unlikePost(postNum):
    
    # find unlike button container
    unlike_btn_container = WebDriverWait(driver,20).until(EC.presence_of_all_elements_located(('tag name','section')))[2]
    # get like/unlike button
    btn = unlike_btn_container.find_element('tag name','svg')
    if btn.get_attribute('aria-label')=='Unlike': # if liked , unlike post
        btn.click()  
        print(f'post {postNum} unliked')

    else:      # if posts is already unliked
        print(f'post {postNum} is already Unliked') 
        
    # close the post
    closeBtn = [i for i in WebDriverWait(driver,20).until(EC.presence_of_all_elements_located(('tag name','svg')))
     if(i.get_attribute('aria-label')=='Close')
    ][0]

    closeBtn.click()


In [43]:
def likePost(postNum):
        
    # find like button container
    like_btn_container = WebDriverWait(driver,20).until(EC.presence_of_all_elements_located(('tag name','section')))[2]
    # get like/unlike button
    btn = like_btn_container.find_element('tag name','svg')
    if btn.get_attribute('aria-label')=='Like':# if not liked, like posts
        btn.click()  
        print(f'post {postNum} liked')
        
    else: # if already liked
        print(f'post {postNum} is already liked')
    
    # close post
    closeBtn = [i for i in WebDriverWait(driver,20).until(EC.presence_of_all_elements_located(('tag name','svg')))
     if(i.get_attribute('aria-label')=='Close')
    ][0]

    closeBtn.click()


In [44]:
def visitPosts(top,action):
    
    i= 1
    # find posts conainer
    posts_container = WebDriverWait(driver,20).until(EC.visibility_of_element_located(('tag name','article')))
    
    # get all posts from posts container
    posts =WebDriverWait(posts_container,10).until(EC.presence_of_all_elements_located(('tag name','a')))
    posts_count=len(posts) #keeps count of fetched posts
    
    while i<=top:
        posts[i].click() #click ith post
        
        #like/unlike posts based on action
        if action=='like':
            likePost(i)
        
        elif action=='unlike':
            unlikePost(i)
        
        # if all posts are traversed , get newly loaded posts
        if i==posts_count-1:
            posts =posts + WebDriverWait(posts_container,10).until(EC.presence_of_all_elements_located(('tag name','a')))
            posts = list(OrderedSet(posts))
            posts_count=len(posts)

        i+=1


# part(1):- Like top 30 posts of 'dilsefoodie'

In [ ]:
profile_name = 'dilsefoodie'
searchAndOpenProfile(profile_name)  # search and open profile
time.sleep(3)
visitPosts(30,'like')  # visit and like top 30 posts


# part(2):- Unlike top 30 posts of 'dilsefoodie'

In [ ]:
visitPosts(30,'unlike') # visit and unlike top 30 posts

# 6. Extract List of Followers

In [236]:
def extractFollowersList(profile_name,count):

    searchAndOpenProfile(profile_name) # search and open profile 
    time.sleep(3)

    # find and click followers button
    followersBtn = driver.find_element('partial link text','followers')
    followersBtn.click()
    
    followersList=[]   
    
    # fetch usernames webElement
    usernames = WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located(('class name','_aad7')))

    followersBox = WebDriverWait(driver,20).until(EC.visibility_of_element_located(('class name','_aano'))) # contains followers list
    current_height = driver.execute_script("return arguments[0].scrollHeight;",followersBox) # current height of followers box
    i=0
    
    while i<count:

        if(i==len(usernames)):  # if traversed all followers in usernames webElement

            # scroll to get more followers 
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", followersBox)
            time.sleep(2.5)
            new_height = driver.execute_script("return arguments[0].scrollHeight;",followersBox) # get new height

            # if reached end of followers box break loop
            if current_height==new_height:
                break
            
            # update current height with new height of followersBox
            current_height = new_height

            usernames=WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located(('class name','_aad7')))

        followersList.append(usernames[i].text) # insert username in followers list
        i+=1

    # if i < count (case when view to followers list is restricted)
    if i<count:
        print(f'\n\n\t allowed to see only {len(usernames)} followers ')

    return followersList


# part(1):- extract first 500 followers of 'foodtalkindia' and 'sodelhi'

In [238]:
foodtalkindia_followers = extractFollowersList('foodtalkindia',500) # get first 500 folloers of foodtalkindia
print('Foodtalkindia followers:-')
print(foodtalkindia_followers)

print('\n\n')

sodelhi_followers = extractFollowersList('sodelhi',500) # get first 500 followers of sodelhi
print('Sodelhi followers:-')
print(sodelhi_followers) 


# part(2):- print all the followers of 'foodtalkindia' that you are following but those who don't follow you

In [ ]:
driver.get('https://www.instagram.com') # open home page
userProfileLink = driver.find_element('xpath','/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[1]/div/div/div/div/div[2]/div[8]/div/span/div/a')
myUsername = userProfileLink.get_attribute('href')[25:].strip('/') # get username of user using userprofilelink

for follower in foodtalkindia_followers:
    
    searchAndOpenProfile(follower) # open this followers profile page

    # get following / follow button
    btn = WebDriverWait(driver, 5).until(EC.presence_of_element_located(("xpath","/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/div[2]/section/main/div/header/section/div[1]/div[1]/div/div[1]/button")))

    if(btn.text=='Following'):

        # open followers list of this follower
        followingBtn = WebDriverWait(driver,20).until(EC.presence_of_element_located(('partial link text','following')))
        followingBtn.click()
            
        time.sleep(2)
        
        # get firstFollower of this follower
        firstFollower = WebDriverWait(driver,20).until(EC.visibility_of_element_located(('class name','_aad7')))
        
        if(myUsername!=firstFollower.text):# if name of firstFollower does not match my username print follower
            print(follower)


# 7. check the story of coding.ninjas.com

In [228]:
searchAndOpenProfile('coding.ninjas')  # open coding ninjas profile
time.sleep(3)

storyButton = driver.find_element('class name','_aarf') # get story Button

if storyButton.get_attribute('aria-disabled') == 'false': # if story present
    
    canvas = driver.find_element('tag name','canvas')  # find container of profile image
    style = canvas.get_attribute('style').replace(' ','').split(';') # fetch style attribute
    
    height = style[3].split(":")[1]  # store height and width
    width = style[4].split(':')[1]

    if height=='168px' and width=='168px':  # if condition true story is unseen , see story

        print("Viewing the story...")
        
        storyButton.click();  # open stories  
        time.sleep(1.5)
        
        while True:
            
            try:
                # find next story button and click
                next_story_button =driver.find_element('class name','_ac0d')  
                next_story_button.click()
                time.sleep(1)
                
            except NoSuchElementException : # if next story button not found break loop
                break

    else:
        print('Story already seen')
    
else: # no stories
    print('no stories available')


Story already seen
